### Modal Analysis

In [8]:
import numpy as np
from numpy.linalg import *
import argparse, os
import time
from configparser import ConfigParser
from scipy.linalg import eigh
from scipy.sparse import coo_matrix
import scipy.sparse

from scipy.io.wavfile import write


def getSignedTetVolume(ele_points):
    point0 = np.resize(ele_points[0], (3, 1))
    point1 = np.resize(ele_points[1], (3, 1))
    point2 = np.resize(ele_points[2], (3, 1))
    point3 = np.resize(ele_points[3], (3, 1))
    Dm = np.hstack((point0 - point3, point1 - point3, point2 - point3))
    volume = det(Dm) / 6
    return volume

def getMeshInfo_vtk(filename):
    import meshio
    mesh = meshio.read(filename)
    mesh_points = mesh.points.tolist()
    mesh_elements = mesh.cells[0].data.tolist()
    return mesh_points, mesh_elements


class ModalAnalysis:

    def __init__(self) -> None:
        pass

    def setVtkFile(self, vtk_file):
        print("[ INFO] Reading mesh info...")
        self.vtk_filepath = vtk_file
        self.mesh_points, self.mesh_elements = getMeshInfo_vtk(self.vtk_filepath)
        self.num_vtx = len(self.mesh_points)
        print('[ INFO] done')

    def setMaterial(self, material_file):
        # print("[ INFO] Reading material file...", material_file)
        cp = ConfigParser()
        cp.read(material_file, 'utf-8')
        self.material = {}
        for key in cp['DEFAULT']:
            if(key != 'name'):
                self.material[key] = float(cp['DEFAULT'][key])
        # print('[ INFO] done')

    def setOutputPath(self, output_path):
        self.output_path = output_path


    def constructM_ori(self):
        # print("[ INFO] Generating M ori matrix...")
        M_ori = np.zeros((3 * self.num_vtx, 3 * self.num_vtx))
        for ele, ele_pts_idx in enumerate(self.mesh_elements):
            ele_pts_pos = [self.mesh_points[ele_pts_idx[p]] for p in range(4)]
            volume = abs(getSignedTetVolume(ele_pts_pos))
            for i in range(4):
                for j in range(4):
                    for k in range(3):
                        I = ele_pts_idx[i]
                        J = ele_pts_idx[j]
                        M_ori[3*I+k][3*J+k] += 0.05 * self.material['density'] * volume * ( 1 + (i == j))

            # if(ele % 50 == 0):
                # print("at element ", ele)
        self.M_ori = M_ori
        self.M_coo = coo_matrix(M_ori)
        # print('[ INFO] done')

    def constructK_ori(self):
        # print("[ INFO] Generating K ori matrix...")
        K_ori = np.zeros((3 * self.num_vtx, 3 * self.num_vtx))
        for ele, ele_pts_idx in enumerate(self.mesh_elements):
            ele_pts_pos = [self.mesh_points[ele_pts_idx[p]] for p in range(4)]

            # k_i = getElementStiffness(ele_pts_pos, self.material['youngs'], self.material['poisson'])

            youngs = self.material['youngs']
            poisson = self.material['poisson']

            # same as getElementStiffness
            lambda_ = youngs * poisson / (1 + poisson) / (1 - 2 * poisson)
            mu_ = youngs * 0.5 / (1 + poisson) 
            mb = np.zeros((4, 4))
            for i in range(4):
                mb[0][i] = ele_pts_pos[i][0]
                mb[1][i] = ele_pts_pos[i][1]
                mb[2][i] = ele_pts_pos[i][2]
                mb[3][i] = 1
            beta_ = np.linalg.inv(mb)

            volume = abs(getSignedTetVolume(ele_pts_pos))
            
            for i in range(4):
                for j in range(4):
                    for a in range(3):
                        for b in range(3):
                            value = lambda_ * beta_[i][a] * beta_[j][b]
                            if ( a == b ):
                                sum_ = 0
                                for k in range(3):
                                    sum_ += beta_[i][k] * beta_[j][k]
                                value += mu_ * sum_
                            value *= 0.5 * volume

                            I = ele_pts_idx[i]
                            J = ele_pts_idx[j]
                            K_ori[3*I+a][3*J+b] += value
            # if(ele % 50 == 0):
                # print("at element ", ele)
        self.K_ori = K_ori
        self.K_coo = coo_matrix(K_ori)
        # print('[ INFO] done')

    def ged(self, k = 50):
        from scipy.sparse.linalg import eigsh
        self.evals, self.evecs = eigsh(A = self.K_coo, M = self.M_coo, which='LM', sigma=0, k=k)

    def saveMK_npz(self):
        print('[ INFO] saving Mass & Stiff matrix to' + self.output_path)
        # np.savez(os.path.join(self.output_path, 'dense_MK.npz'), mass=self.M_ori, stiff=self.K_ori)
        scipy.sparse.save_npz(os.path.join(self.output_path, './M_coo.npz'), self.M_coo)
        scipy.sparse.save_npz(os.path.join(self.output_path, './K_coo.npz'), self.K_coo)
        print('[ INFO] done')

    def saveEigen(self):
        print('[ INFO] saving Mass & Stiff matrix to' + self.output_path)
        np.savez(os.path.join(self.output_path, 'eigen.npz'), evals=self.evals, evecs=self.evecs)


In [217]:
material_id = 0

filename = 'bottle11'
outputpath = os.path.join('./EXP/modelnet/', filename, str(material_id))
inputpath = os.path.join('./EXP/modelnet/', filename, filename+'.vtk')

In [218]:
ma = ModalAnalysis()
material_path = '../material/material-{}.cfg'.format(material_id)
ma.setVtkFile(inputpath)
ma.setMaterial(material_path)
ma.setOutputPath(outputpath)

TIME_0 = time.time()
ma.constructM_ori()
TIME_1 = time.time()
ma.constructK_ori()
TIME_2 = time.time()
ma.ged(k=100)
TIME_3 = time.time()
ma.saveEigen()

print('[ PROFILE] M  ', TIME_1 - TIME_0)
print('[ PROFILE] K  ', TIME_2 - TIME_0)
print('[ PROFILE] GED', TIME_3 - TIME_0)

[ INFO] Reading mesh info...
[ INFO] done
[ INFO] saving Mass & Stiff matrix to./EXP/modelnet/bottle11\0
[ PROFILE] M   2.123480796813965
[ PROFILE] K   8.219257354736328
[ PROFILE] GED 9.864937543869019


### Load Eigen values and vectors

In [12]:
import os
import numpy as np


# return ksi, omegas, omega_d, freqs
def calFreqs(evals, beta, alpha):
    num_modes = len(evals)
    valid_map = np.zeros(num_modes)
    omegas = np.zeros(num_modes)
    omega_d = np.zeros(num_modes)
    ksi = np.zeros(num_modes)
    freqs = np.zeros(num_modes)

    for i in range(num_modes):
        if (evals[i] < 0):
            valid_map[i] = 0
            print('evals < 0 at ', i)
            continue

        omegas[i] = np.sqrt(evals[i])

        if (omegas[i] < 100 or omegas[i] > 2e5):
            print(f'omegas[{i}] = {omegas[i]} is out of 20hz 20000hz range')
            valid_map[i] = 0
            continue
        
        ksi[i] = (beta + alpha * evals[i]) / 2 / omegas[i]
        scale = 1 - ksi[i] * ksi[i]
        if (scale < 0 ):
            valid_map[i] = 0
            print('1 - ksi^2 < 0 at', i)
            continue

        omega_d[i] = omegas[i] * np.sqrt(scale)
        freqs[i] = 0.5 * omega_d[i] / np.pi
    return ksi, omegas, omega_d, freqs

# return mode_sample, samples
def genSound(ksi, omegas, omega_d, activation, fs, duration):
    num_modes = len(ksi)

    time_slot = np.arange(fs * duration) / fs

    mode_sample = np.zeros((num_modes, len(time_slot)))
    samples = np.zeros(len(time_slot))

    for i in range(num_modes):
        if(omega_d[i] != 0):
            amplitude = np.exp(time_slot * (-1) * ksi[i] * omegas[i]) * abs(activation[i]) / omega_d[i]
            mode_sample[i] = (np.sin(omega_d[i] * time_slot ) * amplitude).astype(np.float32)
            samples += mode_sample[i]
    return mode_sample, samples

def saveSound(filename, fs, samples):
    from scipy.io.wavfile import write
    tmp_samples = samples * 1e6
    write(filename, fs, tmp_samples)

In [219]:
eigen_file = np.load(os.path.join(outputpath, 'eigen.npz'))
evals = eigen_file['evals']
evecs = eigen_file['evecs']

BETA = ma.material['beta']
ALPHA = ma.material['alpha']
print(BETA, ALPHA)
ksi, omegas, omega_d, freqs = calFreqs(evals, beta=BETA, alpha=ALPHA)

5.0 1e-07
evals < 0 at  0
omegas[1] = 0.00103909231060257 is out of 20hz 20000hz range
omegas[2] = 0.0023379927381603806 is out of 20hz 20000hz range
omegas[72] = 201066.44608752136 is out of 20hz 20000hz range
omegas[73] = 201611.5350315079 is out of 20hz 20000hz range
omegas[74] = 202160.3347766367 is out of 20hz 20000hz range
omegas[75] = 204079.04803648542 is out of 20hz 20000hz range
omegas[76] = 204620.0056847695 is out of 20hz 20000hz range
omegas[77] = 210845.7865653752 is out of 20hz 20000hz range
omegas[78] = 211308.2902489214 is out of 20hz 20000hz range
omegas[79] = 212097.8916148654 is out of 20hz 20000hz range
omegas[80] = 215261.97495844623 is out of 20hz 20000hz range
omegas[81] = 216149.72176216677 is out of 20hz 20000hz range
omegas[82] = 217264.00266630814 is out of 20hz 20000hz range
omegas[83] = 218141.9174594845 is out of 20hz 20000hz range
omegas[84] = 218694.66582076764 is out of 20hz 20000hz range
omegas[85] = 219673.93575119346 is out of 20hz 20000hz range
ome

In [220]:
ksi_2 = ksi
omegas_2 = omegas
omega_d_2 = omega_d
freqs_2 = freqs

In [221]:
contact_pos = 8
contact_force = [0, 0, 5]
activation = np.zeros(100)
for dir in range(3):
    activation += contact_force[dir] * evecs[3*contact_pos + dir]

### Post-process

In [73]:
Ceramic =   2700,   7.4E10, 0.19,   5,  1E-7
Glass =     2600,   6.2E10, 0.20,   1,  1E-7
Wood =      750,    1.1E10, 0.25,   60, 2E-6
Plastic =   1070,   1.4E9,  0.35,   30, 1E-6
Iron =      8000,   2.1E11, 0.28,   5,  1E-7
Polycarb =  1190,   2.4E9,  0.37,   0.5,4E-7
Steel =     7850,   2.0E11, 0.29,   5,  3E-8
Tin =       7265,   5e10,   0.325,  2,  3E-8

def omega_rate(src, target, rescale):
    k1 = target[1] / src[1]
    k2 = target[0] / src[0]
    return k1**0.5*k2**(-0.5)/rescale

def activation_rate(src, target, rescale):
    k1 = target[1] / src[1]
    k2 = target[0] / src[0]
    return k2**(-0.5)*rescale**(-3/2)

In [212]:
rescale = 1
rate_freq = omega_rate(Ceramic, Plastic, rescale)
rate_act = activation_rate(Ceramic, Plastic, rescale)
print(rate_freq, rate_act)

0.21849331365778657 1.5885101466409677


In [222]:
duration = 2
fs = 44100
_, samples = genSound(ksi, omegas, omega_d, activation, fs, duration)
# _, samples = genSound(ksi, omegas, omega_d * rate_freq, activation * rate_act, fs, duration)
# _, post_samples_1 = genSound(ksi_2, omegas, omega_d * rate_freq, activation * rate_act, fs, duration)
# _, post_samples_2 = genSound(ksi_2, omegas_2, omega_d * rate_freq, activation * rate_act, fs, duration)

### Save Sound

In [223]:
saveSound(os.path.join(outputpath, 'p'+str(contact_pos) +'.wav'), fs, samples)
# saveSound(os.path.join(outputpath, 'p'+str(contact_pos) +'_post_1.wav'), fs, post_samples_1)
# saveSound(os.path.join(outputpath, 'p'+str(contact_pos) +'_post_2.wav'), fs, post_samples_2)